https://towardsdatascience.com/knowing-these-you-can-cover-99-of-file-operations-in-python-84725d82c2df

# Нативные способы создать, открыть, прочитать и закрыть файлы

In [4]:
file = open("file1.txt","w+")  # w+ означает запись
file.write("a new line")  # запишем текст
file.close()  # закроем файл

Основные методы работы с файлами:

r	- чтение (по умолчанию)
r+	- и чтение и запись (находимся в начале самого файла)
w	- запись (обрезает файл, если такой файл уже существует)
w+	- открыть для записи и чтения (обрезает файл, если такой файл уже существует)
a	- открываем для добавления новых данных (добавляет данные, если файл существует; начинает с конца файла)

In [5]:
# Открываем файлы правильно
with open("file1.txt", "a") as file1:
    file1.write("\nthe second line")

Однако, открытие при помощи контекстного менеджера сопряжено с потерей производительности.
Все дело в том, что мы каждый раз закрываем и открываем файл.
С другой стороны, это сильно помогает в случае каких-то непредвиденных ошибок.
Наши данные будут надежно сохранены в файле.

In [7]:
def _write_to_file(file, line):
    with open(file, "a") as f:
        f.write(line)

def _valid_records():
    for i in range(100000):
        if i % 2 == 0:
            yield i

def use_context_manager_2(file):
    for line in _valid_records():
        _write_to_file(file, str(line))

def use_context_manager_1(file):
    with open(file, "a") as f:
        for line in _valid_records():
            f.write(str(line))

def use_close_method(file):
    f = open(file, "a")
    for line in _valid_records():
        f.write(str(line))
    f.close()

%time use_close_method("test.txt")
%time use_context_manager_1("test.txt")
%time use_context_manager_2("test.txt")

CPU times: user 51.1 ms, sys: 6.12 ms, total: 57.3 ms
Wall time: 66.8 ms
CPU times: user 43.1 ms, sys: 3.89 ms, total: 46.9 ms
Wall time: 53.3 ms
CPU times: user 2.39 s, sys: 4.25 s, total: 6.64 s
Wall time: 8.29 s


Существует 3 способа прочитать данные в файле:
- read()
- readline()
- readlines()